In [1]:
# Autosave each 30s
%autosave 30

import numpy as np
import pandas as pd
import pickle as pkl
import tensorflow as tf
import os
import timeit
import copy
import gc
import math
import time
import scipy

from scipy import stats
from scipy.special import erfc
from scipy.signal import fftconvolve
from itertools import product, chain, combinations
from collections import defaultdict, Counter, OrderedDict
from tqdm import tqdm

#sklearn
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import CountVectorizer

#skimage
import skimage
from skimage import io, transform, color
from skimage.transform import resize

#IPython imports
from IPython.display import Image, display, clear_output
from IPython.core.display import HTML
from IPython.display import FileLink, FileLinks

#matplotlib
import matplotlib
import matplotlib as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter
%matplotlib inline

#https://matplotlib.org/users/customizing.html
matplotlib.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
matplotlib.rcParams['legend.handletextpad'] = 0.01
matplotlib.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
matplotlib.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.titlesize'] = 20
# Configuring latex fonts
matplotlib.rc('font', **{'family':'serif'})
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', unicode=True)
matplotlib.rc('text.latex', preamble=r'\usepackage[utf8]{inputenc}')
matplotlib.rc('text.latex', preamble=r'\usepackage[english]{babel}')
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')

%load_ext autoreload
%autoreload 1
# Below comes the list of modules which is automatically reimported
# %aimport cs231n.classifiers.k_nearest_neighbor

Autosaving every 30 seconds


In [2]:
import sys
library_path = '../../../'
if library_path not in sys.path:
    sys.path.append(library_path)
    
CIFAR10_DIR = '../../../datasets/CIFAR10'

from ml.utils import get_CIFAR10_data, load_CIFAR10
from ml.generators import BatchGenerator
from ml.utils.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from ml.neural_network.cs231n.second.layers import *
from ml.neural_network.cs231n.second.optimizers import *
from ml.neural_network.cs231n.second.solver import Solver
from ml.neural_network.cs231n.second.fcn import FullyConnectedNet
from ml.neural_network.cs231n.second.two_layer_net import TwoLayerNet

from ml.neural_network.cs231n.second.conv_layers import *
from ml.neural_network.cs231n.second.conv_layers_fast import *

#from ml.linear_model import svm_loss_naive, svm_loss_vectorized, softmax_loss_naive, softmax_loss_vectorized
#from ml.neural_network import Sequential, NetworkTrainer
#from ml.neural_network.layers import Dense, Dropout, SoftMax
#from ml.neural_network.nonlinearities import ReLU, LeakyReLU, ELU, SoftPlus
#from ml.neural_network.criterions import MulticlassLogLoss

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

<a id='toc'></a>
# Table of Contents

* [2. Convolutional Networks](#cnn)
    * [2.1 Convolution](#cnn_conv)
        * [2.1.1 Convolution: Naive forward pass](#cnn_conv_forw)
        * [2.1.2 Aside: Image processing via convolutions](#cnn_conv_aside)
        * [2.1.3 Convolution: Naive backward pass](#cnn_conv_back)
    * [2.2 Max pooling](#max_pooling)
        * [2.2.1 Max pooling: Naive forward](#cnn_max_pooling_forw)
        * [2.2.2 Max pooling: Naive backward](#cnn_max_pooling_back)
    * [2.3 Fast layers](#cnn_fast)
    * [2.4 Convolutional "sandwich" layers](#cnn_sand)
    * [2.5 Spatial Batch Normalization](#cnn_spatial)
        * [2.5.1 Spatial batch normalization: forward](#cnn_spatial_forw)
        * [2.5.2 Spatial batch normalization: backward](#cnn_spatial_back)
    * [2.6 Three Layer ConvNet](#cnn_three)
        * [2.6.1 Sanity check loss](#cnn_three_sanity)
        * [2.6.2 Gradient check](#cnn_three_grad)
        * [2.6.3 Overfit small data](#cnn_three_overfit)
        * [2.6.4 Train net](#cnn_three_train)
        * [2.6.5 Visualize filters](#cnn_three_visualize)